In [1]:
!pip install tensorflow

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import cv2
import numpy as np
import zipfile
from sklearn.model_selection import train_test_split

In [4]:
zip_file_path = '/content/drive/MyDrive/test_detection_app/augmented_images_apple.zip'
extraction_path = 'apple_data_folder'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [5]:
zip_file_path = '/content/drive/MyDrive/test_detection_app/augmented_images_banana.zip'
extraction_path = 'banana_data_folder'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [6]:
apple_path = '/content/apple_data_folder/appl_augmented_images'
banana_path = '/content/banana_data_folder/banana_augmented_images'

In [7]:
def load_and_preprocess_images(folder_path, label, img_size=(28, 28)):
    images = []
    labels = []

    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        images.append(img)
        labels.append(label)

    return images, labels

In [8]:
apple_images, apple_labels = load_and_preprocess_images(apple_path, label=1)
banana_images, banana_labels = load_and_preprocess_images(banana_path, label=0)

In [13]:
combined_images = np.array(apple_images + banana_images)
combined_labels = np.array(apple_labels + banana_labels)

In [14]:
X_train, X_temp, y_train, y_temp = train_test_split(combined_images, combined_labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [15]:
def inception_module(x, filter_config):
    (filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj) = filter_config

    path1 = layers.Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

    path2 = layers.Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    path2 = layers.Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(path2)

    path3 = layers.Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    path3 = layers.Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(path3)

    path4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    path4 = layers.Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(path4)

    return layers.concatenate([path1, path2, path3, path4], axis=-1)




input_shape = (28, 28, 3)
input_layer = layers.Input(shape=input_shape)

x = inception_module(input_layer, filter_config=(64, 96, 128, 16, 32, 32))

x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(2, activation='softmax')(x)

In [16]:
model = models.Model(inputs=input_layer, outputs=x)

In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))

Epoch 1/15
23/23 [==============================] - 3s 20ms/step - loss: 0.6937 - accuracy: 0.6806 - val_loss: 0.2441 - val_accuracy: 0.9333
Epoch 2/15
23/23 [==============================] - 0s 7ms/step - loss: 0.0586 - accuracy: 0.9958 - val_loss: 0.0060 - val_accuracy: 1.0000
Epoch 3/15
23/23 [==============================] - 0s 8ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 4/15
23/23 [==============================] - 0s 8ms/step - loss: 6.9496e-04 - accuracy: 1.0000 - val_loss: 6.7530e-04 - val_accuracy: 1.0000
Epoch 5/15
23/23 [==============================] - 0s 8ms/step - loss: 2.9477e-04 - accuracy: 1.0000 - val_loss: 4.4234e-04 - val_accuracy: 1.0000
Epoch 6/15
23/23 [==============================] - 0s 8ms/step - loss: 1.6562e-04 - accuracy: 1.0000 - val_loss: 3.1990e-04 - val_accuracy: 1.0000
Epoch 7/15
23/23 [==============================] - 0s 8ms/step - loss: 1.3039e-04 - accuracy: 1.0000 - val_loss: 3.5097e-04 - val_accu

In [19]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

8/8 [==============================] - 0s 3ms/step - loss: 1.3531e-05 - accuracy: 1.0000
Test accuracy: 1.0, Test loss: 1.3530726391763892e-05


In [20]:
model.save("detection_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
model_tfl = tf.keras.models.load_model("detection_model.h5")


converter = tf.lite.TFLiteConverter.from_keras_model(model_tfl)
tflite_model = converter.convert()

In [22]:
with open('detection_model.tflite', 'wb') as f:
  f.write(tflite_model)